In [6]:
from rich import print

from cali._plate_viewer._analysis_new import AnalysisRunner
from cali.readers import TensorstoreZarrReader
from cali.sqlmodel._visualize_experiment import print_experiment_tree

# Run from Database Example

In [7]:
from cali.sqlmodel._json_to_db import load_experiment_from_db

In [ ]:
database_path = (
    "/Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk_analysis/cali.db"
)
exp = load_experiment_from_db(database_path)
print_experiment_tree(exp)

🧪 evoked
├── Imported from /Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk_analysis
├── ⚙️ Analysis Settings available
└── 📋 96-well (96-well)
    └── 🧫 B5 - 🧪 Conditions: c1, t1
        └── 📷 B5_0000_p0 (fov: 0 - pos: 0)
            ├── 🔬 ROI 1 - 🔋 active - ✨ spontaneous
            │   ├── 📊 Trace data available
            │   ├── 📈 Data analysis available
            │   └── 🎭 ROI mask available
            ├── 🔬 ROI 2 - 🔋 active - ⚡️ stimulated
            │   ├── 📊 Trace data available
            │   ├── 📈 Data analysis available
            │   └── 🎭 ROI mask available
            ├── 🔬 ROI 3 - 🔋 active - ⚡️ stimulated
            │   ├── 📊 Trace data available
            │   ├── 📈 Data analysis available
            │   └── 🎭 ROI mask available
            └── 🔬 ROI 4 - 🔋 active - ✨ spontaneous
                ├── 📊 Trace data available
                ├── 📈 Data analysis available
                └── 🎭 ROI mask available

In [4]:
# delete existing rois
for well in exp.plate.wells:
    for fov in well.fovs:
        fov.rois = []

print_experiment_tree(exp)

🧪 evoked
├── Imported from /Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk_analysis
├── ⚙️ Analysis Settings available
└── 📋 96-well (96-well)
    └── 🧫 B5 - 🧪 Conditions: c1, t1
        └── 📷 B5_0000_p0 (fov: 0 - pos: 0)

In [ ]:
runner = AnalysisRunner()
runner.set_experiment(exp)
data = TensorstoreZarrReader(exp.data_path)
# print("data:", data.store)
runner.set_data(data)
# print(runner.get_settings())

def _p(msg: str) -> None:
    print("ANALYSIS INFO:", msg)
runner.analysisInfo.connect(_p)

<function __main__._p(msg: str) -> None>

In [ ]:
runner.run()

2025-11-08 21:08:44,096 - cali_logger - INFO - Starting Analysis...
2025-11-08 21:08:44,098 - cali_logger - INFO - Starting traces analysis...
2025-11-08 21:08:44,099 - cali_logger - INFO - Threads: 5


2025-11-08 21:08:44,117 - cali_logger - INFO - Extracting Traces Data from Well B5_0000_p0.
Extracting Traces Data from Well B5_0000_p0.: 100%|██████████| 4/4 [00:00<00:00, 153.13it/s]
2025-11-08 21:08:44,145 - cali_logger - INFO - Position 0 completed.


ANALYSIS INFO: Position 0 completed.

2025-11-08 21:08:44,147 - cali_logger - INFO - All positions processed.


In [28]:
# print_experiment_tree(exp)
print(exp.plate.wells[0].fovs[0].rois)

🧪 evoked
├── Imported from /Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk_analysis
├── ⚙️ Analysis Settings available
└── 📋 96-well (96-well)
    └── 🧫 B5 - 🧪 Conditions: c1, t1
        └── 📷 B5_0000_p0 (fov: 0 - pos: 0)

# Run From Directories Example

In [8]:
from datetime import datetime

import useq

from cali.sqlmodel._models import AnalysisSettings, Experiment
from cali.sqlmodel._useq_plate_to_db import useq_plate_plan_to_db

In [9]:
database_path = (
    "/Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk_analysis/cali_new.db"
)
data_path = (
    "/Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk.tensorstore.zarr"
)
analysis_path = "/Users/fdrgsp/Desktop/cali_test"
labels_path = "/Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk_labels"

new_exp = Experiment(
    id=0,
    name="New Experiment",
    description="A Test Experiment.",
    created_at=datetime.now(),
    database_path=database_path,
    data_path=data_path,
    labels_path=labels_path,
    analysis_path=analysis_path,
)
print(new_exp)

Experiment(
    id=0,
    name='New Experiment',
    description='A Test Experiment.',
    created_at=datetime.datetime(2025, 11, 8, 22, 26, 53, 253769),
    database_path='/Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk_analysis/cali_new.db',
    data_path='/Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk.tensorstore.zarr',
    labels_path='/Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk_labels',
    analysis_path='/Users/fdrgsp/Desktop/cali_test',
    experiment_type='Spontaneous Activity',
    positions_analyzed=[]
)

In [10]:
# load the data and get the useq plate plan from the sequence
data = TensorstoreZarrReader(new_exp.data_path)
plate_plan = data.sequence.stage_positions
assert isinstance(plate_plan, useq.WellPlatePlan)

# Define plate maps for conditions
plate_maps = {
    "genotype": {"B5": "WT"},
    "treatment": {"B5": "Vehicle"},
}

# Create plate with plate_maps and conditions in one step
plate = useq_plate_plan_to_db(plate_plan, new_exp, plate_maps)
new_exp.plate = plate

print(new_exp.plate)
print_experiment_tree(new_exp)

Plate(
    name='96-well',
    plate_type='96-well',
    rows=8,
    columns=12,
    plate_maps={'genotype': {'B5': 'WT'}, 'treatment': {'B5': 'Vehicle'}},
    experiment_id=0,
    id=None
)

🧪 New Experiment
├── A Test Experiment.
└── 📋 96-well (96-well)
    └── 🧫 B5 - 🧪 Conditions: WT, Vehicle

In [11]:
analysis_settings = AnalysisSettings(
    neuropil_inner_radius=0,
    peaks_prominence_multiplier=1.0,
    spikes_sync_cross_corr_lag=5,
    neuropil_min_pixels=0,
    calcium_sync_jitter_window=5,
    led_power_equation="y = 11 * x - 6",
    neuropil_correction_factor=0.0,
    calcium_network_threshold=90.0,
    led_pulse_duration=100,
    led_pulse_powers=[2.0, 4.0, 6.0],
    led_pulse_on_frames=[3, 53, 103],
    decay_constant=0.0,
    spike_threshold_value=1.0,
    stimulation_mask_path="/Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk_analysis/stimulation_mask.tif",
    id=1,
    dff_window=30,
    spike_threshold_mode="multiplier",
    threads=1,
    peaks_height_value=3.0,
    burst_threshold=30.0,
    experiment_id=1,
    peaks_height_mode="multiplier",
    burst_min_duration=3,
    stimulation_mask_id=1,
    created_at=datetime(2025, 11, 8, 20, 25, 1, 911683),
    peaks_distance=2,
    burst_gaussian_sigma=2.0,
)
new_exp.analysis_settings = analysis_settings

print_experiment_tree(new_exp)

🧪 New Experiment
├── A Test Experiment.
├── ⚙️ Analysis Settings available
└── 📋 96-well (96-well)
    └── 🧫 B5 - 🧪 Conditions: WT, Vehicle

In [17]:
runner = AnalysisRunner()
runner.set_experiment(new_exp)
data = TensorstoreZarrReader(new_exp.data_path)
# print("data:", data.store)
runner.set_data(data)
# print(runner.get_settings())


def _p(msg: str) -> None:
    print("ANALYSIS INFO:", msg)
runner.analysisInfo.connect(_p)

<function __main__._p(msg: str) -> None>

In [15]:
runner.run()

2025-11-08 22:28:17,117 - cali_logger - INFO - Starting Analysis...
2025-11-08 22:28:17,118 - cali_logger - INFO - Starting traces analysis...
2025-11-08 22:28:17,119 - cali_logger - INFO - Threads: 5
2025-11-08 22:28:17,120 - cali_logger - INFO - All positions processed.
